# TorchUtils
> Some handy utilities for pytorch 

In [ ]:
#| default_exp torchutils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from torch.utils.data import Dataset
import torch

In [ ]:
#| export
def device_by_name(name: str) -> torch.device:
    ''' Return reference to cuda device by using Part of it's name

        Args:
            name: part of the cuda device name (shuuld be distinct)

        Return:
            Reference to cuda device

        Updated: Yuval 12/10/19
    '''
    assert torch.cuda.is_available(), "No cuda device"
    device = None
    for i in range(torch.cuda.device_count()):
        dv = torch.device("cuda:{}".format(i))
        if name in torch.cuda.get_device_name(dv):
            device = dv
            break
    assert device, "device {} not found".format(name)
    return device

In [ ]:
show_doc(device_by_name)

---

### device_by_name

>      device_by_name (name:str)

Return reference to cuda device by using Part of it's name

Args:
    name: part of the cuda device name (shuuld be distinct)

Return:
    Reference to cuda device

Updated: Yuval 12/10/19

#### How to use

In [ ]:
device_by_name("Tesla")

device(type='cuda', index=0)

If the device doesn't exist we should get an error 

In [ ]:
error = False
try:
    device_by_name("fff")
except AssertionError:
    error = True
assert error

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()